In [1]:
# Importing Libraries
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical


In [2]:
#loading dataset
# Dataset path
base_dir = r'D:\year 3\image processing\Fruits-Recognition\Dataset\fruits-360_dataset_100x100\fruits-360'
print("Base directory exists:", os.path.exists(base_dir))
train_dir = os.path.join(base_dir, 'Training')
test_dir = os.path.join(base_dir, 'Test')
print("Train directory exists:", os.path.exists(train_dir))
print("Test directory exists:", os.path.exists(test_dir))

# List fruit types from the dataset directory
fruit_types = os.listdir(train_dir)  # This will give you the folder names for each fruit type
print("Fruit types in dataset:", fruit_types)

Base directory exists: True
Train directory exists: True
Test directory exists: True
Fruit types in dataset: ['Apple 6', 'Apple Braeburn 1', 'Apple Crimson Snow 1', 'Apple Golden 1', 'Apple Golden 2', 'Apple Golden 3', 'Apple Granny Smith 1', 'Apple hit 1', 'Apple Pink Lady 1', 'Apple Red 1', 'Apple Red 2', 'Apple Red 3', 'Apple Red Delicious 1', 'Apple Red Yellow 1', 'Apple Red Yellow 2', 'Apricot 1', 'Avocado 1', 'Avocado ripe 1', 'Banana 1', 'Banana Lady Finger 1', 'Banana Red 1', 'Beetroot 1', 'Blueberry 1', 'Cabbage white 1', 'Cactus fruit 1', 'Cantaloupe 1', 'Cantaloupe 2', 'Carambula 1', 'Carrot 1', 'Cauliflower 1', 'Cherry 1', 'Cherry 2', 'Cherry Rainier 1', 'Cherry Wax Black 1', 'Cherry Wax Red 1', 'Cherry Wax Yellow 1', 'Chestnut 1', 'Clementine 1', 'Cocos 1', 'Corn 1', 'Corn Husk 1', 'Cucumber 1', 'Cucumber 3', 'Cucumber Ripe 1', 'Cucumber Ripe 2', 'Dates 1', 'Eggplant 1', 'Eggplant long 1', 'Fig 1', 'Ginger Root 1', 'Granadilla 1', 'Grape Blue 1', 'Grape Pink 1', 'Grape Whi

In [3]:
# Resize images to 100x100
image_size = (100, 100)

# Set up Image Augmentation for Training Data
datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize the images
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Use flow_from_directory to load images in batches
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=16,  # Reduce batch size to 16
    class_mode='categorical',
    shuffle=True
)

test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=16,  # Reduce batch size to 16
    class_mode='categorical',
    shuffle=False
)




Found 70491 images belonging to 141 classes.
Found 23619 images belonging to 141 classes.


In [4]:
# Function to load and preprocess images
def load_images_and_labels(dataset_dir):
    images = []
    labels = []
    
    # Enumerate through fruit types (folders in the dataset)
    for label, fruit in enumerate(fruit_types):
        fruit_folder = os.path.join(dataset_dir, fruit)
        for image_name in os.listdir(fruit_folder):
            # Load each image
            img_path = os.path.join(fruit_folder, image_name)
            img = cv2.imread(img_path)
            if img is None:
                continue
            img = cv2.resize(img, image_size)  # Resize image to 100x100
            img = img / 255.0  # Normalize the pixel values to [0, 1]
            
            images.append(img)
            labels.append(label)
    
    images = np.array(images)
    labels = np.array(labels)
    
    return images, labels


In [5]:
# Load images and labels
images, labels = load_images_and_labels(train_dir)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Rescale labels using one-hot encoding
y_train = to_categorical(y_train, num_classes=len(fruit_types))
y_test = to_categorical(y_test, num_classes=len(fruit_types))

In [6]:
# Flatten images for SVM (converting 2D images to 1D vectors)
X_train_flattened = X_train.reshape(X_train.shape[0], -1)
X_test_flattened = X_test.reshape(X_test.shape[0], -1)

# Normalize the data

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_flattened)
X_test_scaled = scaler.transform(X_test_flattened)

# Train SVM Classifier
svm_classifier = SVC(kernel='linear', random_state=42)
svm_classifier.fit(X_train_scaled, np.argmax(y_train, axis=1))

# Predict on test set
y_pred = svm_classifier.predict(X_test_scaled)

# Evaluate the model
accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')


MemoryError: Unable to allocate 12.6 GiB for an array with shape (56392, 30000) and data type float64

In [ ]:
# Visualize augmented images
sample_img = X_train[0]  # Taking one image from training data
sample_img = np.expand_dims(sample_img, axis=0)  # Adding batch dimension

# Generate augmented images
augmented_images = datagen.flow(sample_img)

# Display the augmented images
plt.figure(figsize=(10, 10))
for i in range(9):  # Displaying 9 augmented images
    plt.subplot(3, 3, i+1)
    plt.imshow(augmented_images.next()[0].astype('uint8'))
    plt.axis('off')
plt.show()